# Środowiska ciągłe
### Katarzyna Stępień, Alicja Wójcik, Kacper Stankiewicz

In [1]:
# for autoformatting
# %load_ext jupyter_black

In [1]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 KB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.0 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 KB 15.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.7 MB/s 

## Imports

In [2]:
import gymnasium as gym
import numpy as np

In [3]:
from stable_baselines3 import PPO

In [4]:
from stable_baselines3.ppo.policies import MlpPolicy
from gymnasium import Env
from stable_baselines3.common.base_class import BaseAlgorithm

## Uczenie modelu

W projekcie wykorzystaliśmy ciągłą wersję środowiska MountainCar.

MDP Mountain Car to deterministyczne środowisko Markowa (MDP) 2D, w którym umieszczony jest samochód na dole sinusoidalnej doliny. Jedyne dostępne akcje to przyspieszenia, które można zastosować do samochodu w dwóch dostępnych kierunkach (przód-tył samochodu). Celem MDP jest strategiczne przyspieszenie samochodu, tak, aby osiągnąć stan docelowy na szczycie prawego wzgórza. W bibliotece Gymnasium istnieją dwie wersje tego środowiska: jedna z dyskretnymi akcjami i druga z ciągłymi. Wersja użyta w tym projekcie korzysta z ciągłych akcji.

In [5]:
env = gym.make("MountainCarContinuous-v0")

Do usprawnienia manipulacji hiperparametrów stworzyliśmy klasę pomocniczą, z domyślnymi wartościami parametrów ustawionymi według rekomendacji [repozytorium RL Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

In [6]:
from typing import Dict
from dataclasses import dataclass, asdict, field

@dataclass
class Params:
  normalize_advantage: bool = True
  policy: str = "MlpPolicy"
  batch_size: int = 32
  n_steps: int = 32
  gamma: float = 0.9999
  learning_rate: float = float(7.77e-05)
  ent_coef: float = 0.00429
  clip_range: float = 0.1
  n_epochs: int = 100
  gae_lambda: float = 0.9
  max_grad_norm: int = 5
  vf_coef: float = 0.19
  use_sde: bool = True
  policy_kwargs: Dict = field(default_factory=lambda: dict(log_std_init=-3.29, ortho_init=False))

W bibliotece stable_baselines dostępna jest już wyspecjalizowana funkcja do ewaluacji efektów uczenia modelu. Na potrzeby zadania wykorzystaliśmy ją do zbierania statystyk.

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

In [9]:
import matplotlib.pyplot as plt
import time

def evaluate_and_plot(model: BaseAlgorithm, env: Env, n_eval_episodes: int, warn: bool, deterministic:bool, repeat: int):
  rewards = []
  std_rewards = []
  total_time = 0
  for i in range(repeat):
    start_time = time.time()
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes, deterministic)
    end_time = time.time() - start_time
    rewards.append(mean_reward)
    std_rewards.append(std_reward)
    print(f"Iteration: {i+1}, Evaluation time: {end_time}")
    total_time += end_time
  print(f"Total evaluation time: {total_time}")

  plt.figure(figsize=(10, 5))  # Set the figure size
  y, x = zip(*[(reward, index) for index, reward in enumerate(rewards)])

  plt.plot(list(x), list(y), "-b", label="mean reward")
  plt.plot(list(x), std_rewards, "-r", label="std deviation")
  plt.legend(loc="upper left")

  plt.xlabel('Episode')
  plt.ylabel('Reward')

  plt.title('Rewards per Episode - Mountain cart continuous PPO')
  plt.savefig('rewards_plot_ppo.png')
  plt.show()


### Wizualizacja uczenia - nagranie wideo


In [10]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [11]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

Do "nagrania" wideo wykorzystany został wrapper VecVideoRecorder.

In [12]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

## Uczenie modelu wraz z wizualizacją statystyk oraz nagranie zachowania agenta




In [13]:
def create_model_and_evaluate(params, env, learning_steps, repeat, policy_kwargs = None, filename = "ppo-mountain-car"):
  merged_dict = asdict(params) | {"env": env}
  if policy_kwargs is not None:
    merged_dict = merged_dict | {"policy_kwargs": policy_kwargs}

  model = PPO(**merged_dict)

  start_time = time.time()
  model.learn(total_timesteps=learning_steps)
  end_time = time.time() - start_time
  print(f"Total learning time: {end_time}")

  evaluate_and_plot(model, env, n_eval_episodes=100, warn=False, deterministic=False, repeat=repeat)

  record_video("MountainCarContinuous-v0", model, video_length=2000, prefix=filename)

  return model

### Parametry domyślne

In [ ]:
params = Params()
create_model_and_evaluate(params, env, 50_000, 10, filename="ppo-mountain-car-hipar")

### Różne wariacje hiperparapmetrów

In [ ]:
params = Params()
create_model_and_evaluate(params, env, 50_000, 10, filename="ppo-mountain-car-hipar")

In [ ]:
params = Params()
create_model_and_evaluate(params, env, 50_000, 10, filename="ppo-mountain-car-hipar")

In [ ]:
show_videos("videos", prefix="ppo-mountain-car-hipar")

## Eksperymenty z różnymi architekturami sieci

### Domyślnie wykorzystywana sieć - MlpPolicy

### Modyfikacja pierwsza

In [ ]:
import torch as th

In [ ]:
params = Params()
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[32], vf=[16, 8, 8]))
create_model_and_evaluate(params, env, 1_000, 10, policy_kwargs, filename="ppo-mountain-car-arch")

### Modyfikacja druga

In [ ]:
params = Params()
policy_kwargs = dict(activation_fn=th.nn.Tanh,
                     net_arch=dict(pi=[16, 16, 16], vf=[8, 8]))
create_model_and_evaluate(params, env, 50_000, 10, policy_kwargs, filename="ppo-mountain-car-arch")

## Wizualizacja sieci

**PPO** jest metodą policy gradient, co oznacza, że bezpośrednio optymalizuje politykę agenta. Funkcja polityki agenta to sieć neuronowa, która jest aktualizowana (dość powoli, w celu ograniczenia popełniania złych decyzji przez agenta, prowadzących w zupełnie innym kierunku) na podstawie rezultatów podejmowanych akcji i przyznawanych za nie nagród.

PPO zawiera w sobie dwa modele, aktora i krytyka, o dwóch różnych architekturach, co widać na obrazku niżej:

![ppo](ppo.png)

Nasza początkowa sieć wykorzystuje dwie gęste warstwy w modelu aktora oraz krytyka (jest to wartość domyślna). Wszystkie mają po 64 neurony. Funkcja aktywacji to ReLu.

![ppo64](6464.png)

### Modyfikacja 1

W tym wypadku architektura modelu aktora wygląda następująco:

![actor32](32.png)

Natomiast krytyka:

![critic1688](16881.png)

### Modyfikacja 2

W tym wypadku architektura modelu aktora wygląda następująco:

![actor161616](161616.png)

Natomiast krytyka:

![critic88](881.png)

## Obserwacja zachowania agenta z wyłączonym trybem eksploracji